In [ ]:
%load_ext autoreload
%autoreload 2
from pyrho.core.chargeDensity import ChargeDensity
from pymatgen.io.vasp import Chgcar
from pyrho.core.pgrid import PGrid
from pyrho.core.utils import get_padded_array
from pyrho.core.fourier import PFourier

chgcar = Chgcar.from_hdf5("../test_files/Si.uc.hdf5")
chgcar = ChargeDensity.from_pmg_volumetric_data(chgcar)
chgcar.reorient_axis()

a_mat = chgcar.lattice[:2,:2]
data = chgcar.grid_data[24, :, :]

pg_2D = PGrid(data, a_mat)

In [ ]:
def show_2d(pg_2D):
    av=np.linspace(0,1,pg_2D.grid_data.shape[0],endpoint=False)
    bv=np.linspace(0,1,pg_2D.grid_data.shape[1],endpoint=False)
    AA, BB = np.meshgrid(av,bv,indexing='ij') #indexing to match the lablled array
    xx, yy = np.dot(pg_2D.lattice.T[:2,:2], [AA.flatten(),BB.flatten()])
    xshift, yshift = np.dot(pg_2D.lattice.T[:2,:2], ((av[1]-av[0])/2.,(bv[1]-bv[0])/2.))
    plt.scatter(xx+xshift, yy+yshift, c=np.log(pg_2D.grid_data.flatten()), edgecolors='black',alpha=0.1)
show_2d(pg_2D)


In [ ]:
fft_data = np.abs(np.fft.fftn(pg_2D.grid_data))
pf = PFourier(fft_data, pg_2D.lattice)

def show_2d_f(pf, thresh = 12):
    av=np.linspace(0,1,pf.fourier_data.shape[0],endpoint=False)
    bv=np.linspace(0,1,pf.fourier_data.shape[1],endpoint=False)

    xx, yy = pf.fft_pos_centered_cartesian_s
    mask = pf.fourier_data.flatten() > thresh
    plt.scatter(xx[mask], yy[mask], c=pf.fourier_data.flatten()[mask], edgecolors='black',alpha=0.4)
    plt.axes().set_aspect("equal")


In [ ]:
fft_data = np.abs(np.fft.fftn(pg_2D.grid_data))
pf = PFourier(fft_data, pg_2D.lattice)
show_2d_f(pf)

In [ ]:
g1,g2 = pg_2D.grid_data.shape
pg_super = pg_2D.get_transformed_obj(
    sc_mat=[[1, 1], [1, -2]], frac_shift=[0, 0], grid_out=[g1*3 , g2*2 ], up_sample=2
)
show_2d(pg_super)


In [ ]:
fft_data_super = np.abs(np.fft.fftn(pg_super.grid_data))
pf_super = PFourier(fft_data_super, pg_super.lattice)


In [ ]:
pg_2D.lattice

In [ ]:
show_2d_f(pf_super, thresh = 3*9)

In [ ]:
show_2d_f(pf, thresh = 3)